# CS263 Final Project Notebook 1

## Data pre-processing

Please update the paths before running this notebook. You can get the files here: https://github.com/iangalvez/cs263-vlm-radiology-eval/tree/main

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Graduate/CS263_NLP/FinalProj/

/content/drive/MyDrive/Graduate/CS263_NLP/FinalProj


In [ ]:
!pip install pydicom

In [ ]:
import os, json
names = os.listdir('/content/drive/MyDrive/Graduate/CS263_NLP/FinalProj/MIMIC-CXR-filtered (1)')
for name in names:
  json_file = os.path.join('/content/drive/MyDrive/Graduate/CS263_NLP/FinalProj/MIMIC-CXR-filtered (1)', name, name + '.json')
  with open(json_file, 'r') as json_file:
    report = json.load(json_file)
  print(name)
  print('FINDINGS:',report['FINDINGS'])
  print('INDICATION:',report['INDICATION'])
  print('\n')

In [ ]:
import json
from glob import glob
import os
import shutil
save_root = '/content/drive/MyDrive/Graduate/CS263_NLP/FinalProj/MIMIC-CXR-filtered/'
cases = glob('/content/drive/MyDrive/Graduate/CS263_NLP/FinalProj/MIMIC-CXR-processed/*')
case_ids = os.listdir('/content/drive/MyDrive/Graduate/CS263_NLP/FinalProj/MIMIC-CXR-processed/')
cnt = 0
for i, case in enumerate(cases):
  report_path = os.path.join(case, case_ids[i] + '.json')
  with open(report_path, 'r') as json_file:
    report = json.load(json_file)
  if 'INDICATION' in report.keys() and 'FINDINGS' in report.keys() and 'IMPRESSION' in report.keys():
    shutil.copytree(case, os.path.join(save_root, case_ids[i]))
    cnt += 1
print(cnt)

58


In [ ]:
import json
import re

def txt_to_dict(filename):
  with open(filename, 'r') as file:
      text = file.read()
  record = {}
  text_orig = text.replace("FINAL REPORT", "").strip()
  text = text_orig.split('\n \n')
  # print(text)
  track = 0
  if 'EXAMINATION' in text_orig:
    record['EXAMINATION'] = text[track].split(':')[-1].strip()
    track += 1
  if 'INDICATION' in text_orig:
    record['INDICATION'] = text[track].split(':')[-1].strip()
    track += 1
  if 'TECHNIQUE' in text_orig:
    record['TECHNIQUE'] = text[track].split(':')[-1].strip()
    track += 1
  if 'COMPARISON' in text_orig:
    record['COMPARISON'] = text[track].split(':')[-1].strip()
    track += 1
  if 'FINDINGS' in text_orig:
    record['FINDINGS'] = text_orig.split('FINDINGS:')[-1].strip().split('IMPRESSION:')[0].strip()
    # track += 1
    # record['FINDINGS'] = text[track].strip()
    # track += 1
  if 'IMPRESSION' in text_orig:
    record['IMPRESSION'] = text_orig.split('IMPRESSION:')[-1].strip()
    # track += 1
    # record['IMPRESSION'] = text[track].strip()
    # track += 1
  return record

filename = '/content/drive/MyDrive/Graduate/CS263_NLP/FinalProj/MIMIC-CXR/p10000032/s50414267.txt'
txt_to_dict(filename)

{'EXAMINATION': 'CHEST (PA AND LAT)',
 'INDICATION': '___F with new onset ascites  // eval for infection',
 'TECHNIQUE': 'Chest PA and lateral',
 'COMPARISON': 'None.',
 'FINDINGS': 'There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral\n nodular opacities that most likely represent nipple shadows. The\n cardiomediastinal silhouette is normal.  Clips project over the left lung,\n potentially within the breast. The imaged upper abdomen is unremarkable.\n Chronic deformity of the posterior left sixth and seventh ribs are noted.',
 'IMPRESSION': 'No acute cardiopulmonary process.'}

In [ ]:
import os
import pydicom
from glob import glob
import cv2
import json
import numpy as np
import cv2
import shutil
root = '/content/drive/MyDrive/Graduate/CS263_NLP/FinalProj/MIMIC-CXR/'
save_root = '/content/drive/MyDrive/Graduate/CS263_NLP/FinalProj/MIMIC-CXR-processed/'
os.makedirs(save_root, exist_ok=True)

patients = glob(os.path.join(root, 'p*'))
for patient in patients:
  scans = []
  for f in os.listdir(patient):
    if os.path.isdir(os.path.join(patient, f)):
      scans.append(f)
  for scan in scans:
    # scan_dict = {'imgs': [], 'report': []}
    scan_save_path = os.path.join(save_root, scan)
    os.makedirs(scan_save_path, exist_ok=True)
    imgs_path = os.path.join(save_root, scan, 'imgs')
    os.makedirs(imgs_path, exist_ok=True)

    report_json_path = os.path.join(scan_save_path, scan + '.json')
    print(os.path.join(patient, scan + '.txt'))
    report_dict = txt_to_dict(os.path.join(patient, scan + '.txt'))
    with open(report_json_path, 'w') as json_file:
      json.dump(report_dict, json_file, indent=4)

    scan_path = os.path.join(patient, scan)
    dcms = glob(os.path.join(scan_path, '*.dcm'))
    for dcm in dcms:
      img = pydicom.dcmread(dcm).pixel_array
      img = (img - img.min()) / (img.max() - img.min())
      img = (img * 255).astype(np.uint8)
      H, W = img.shape
      img = cv2.resize(img, (int(W / 4), int(H / 4)))
      cv2.imwrite(os.path.join(imgs_path, os.path.basename(dcm)[:-4] + '.png'), img)

In [ ]:
script_content = """
for i in {10..19}; do
    # Get the list of the first two folders within each pXX directory
    folder_names=$(wget -q -O - "https://physionet.org/files/mimic-cxr/2.0.0/files/p$i/" | \
    grep -oP 'href="p\d+/"' | \
    sed 's/href="//;s|/||' | \
    tail -n 2)

    # Loop over the first two folders and download their contents
    for folder in $folder_names; do
        # Clean up the folder name by removing %22, trailing quotes, and spaces
        clean_folder=$(echo "$folder" | sed 's/%22//g' | sed 's/"$//g' | tr -d '[:space:]')

        # Print the cleaned folder name and URL to check it
        echo "Processing folder: $clean_folder"
        echo "URL: https://physionet.org/files/mimic-cxr/2.0.0/files/p$i/$clean_folder/"

        # Use wget to download the folder
        wget -r -N -c -np -nH --cut-dirs=5 "https://physionet.org/files/mimic-cxr/2.0.0/files/p$i/$clean_folder/"
    done
done
"""

# Write the script content to a file named "download_folders.sh"
with open("download_folders.sh", "w") as file:
    file.write(script_content)

In [ ]:
!chmod +x download_folders.sh
!./download_folders.sh